In [25]:
# Install libraries
!pip install pandas
!pip install nltk
!pip install tensorflow
!pip install scikit-learn
!pip install rouge


[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Import libraries
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Concatenate, Input, concatenate
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [3]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nidhi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nidhi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
tf.config.run_functions_eagerly(True)

# Loading Data & Dataframe Modifications

In [5]:
# Load the song lyrics into a dataframe
songs = pd.read_csv('archive/lyrics-data.csv')
songs.head()

,ALink,SName,SLink,Lyric,language
0,/ivete-sangalo/,Arerê,/ivete-sangalo/arere.html,"Tudo o que eu quero nessa vida,\nToda vida, é\...",pt
1,/ivete-sangalo/,Se Eu Não Te Amasse Tanto Assim,/ivete-sangalo/se-eu-nao-te-amasse-tanto-assim...,Meu coração\nSem direção\nVoando só por voar\n...,pt
2,/ivete-sangalo/,Céu da Boca,/ivete-sangalo/chupa-toda.html,É de babaixá!\nÉ de balacubaca!\nÉ de babaixá!...,pt
3,/ivete-sangalo/,Quando A Chuva Passar,/ivete-sangalo/quando-a-chuva-passar.html,Quando a chuva passar\n\nPra quê falar\nSe voc...,pt
4,/ivete-sangalo/,Sorte Grande,/ivete-sangalo/sorte-grande.html,A minha sorte grande foi você cair do céu\nMin...,pt


In [6]:
# Load the song lyrics into a dataframe
artists = pd.read_csv('archive/artists-data.csv')
artists.head()

,Artist,Genres,Songs,Popularity,Link
0,Ivete Sangalo,Pop; Axé; Romântico,313.0,4.4,/ivete-sangalo/
1,Chiclete com Banana,Axé,268.0,3.8,/chiclete-com-banana/
2,Banda Eva,Axé; Romântico; Reggae,215.0,2.3,/banda-eva/
3,É O Tchan,Axé,129.0,1.6,/e-o-tchan/
4,Claudia Leitte,Pop; Axé; Romântico,167.0,1.5,/claudia-leitte/


In [7]:
lyrics = pd.merge(songs, artists, left_on = 'ALink', right_on = 'Link')
lyrics

,ALink,SName,SLink,Lyric,language,Artist,Genres,Songs,Popularity,Link
0,/ivete-sangalo/,Arerê,/ivete-sangalo/arere.html,"Tudo o que eu quero nessa vida,\nToda vida, é\...",pt,Ivete Sangalo,Pop; Axé; Romântico,313.0,4.4,/ivete-sangalo/
1,/ivete-sangalo/,Se Eu Não Te Amasse Tanto Assim,/ivete-sangalo/se-eu-nao-te-amasse-tanto-assim...,Meu coração\nSem direção\nVoando só por voar\n...,pt,Ivete Sangalo,Pop; Axé; Romântico,313.0,4.4,/ivete-sangalo/
2,/ivete-sangalo/,Céu da Boca,/ivete-sangalo/chupa-toda.html,É de babaixá!\nÉ de balacubaca!\nÉ de babaixá!...,pt,Ivete Sangalo,Pop; Axé; Romântico,313.0,4.4,/ivete-sangalo/
3,/ivete-sangalo/,Quando A Chuva Passar,/ivete-sangalo/quando-a-chuva-passar.html,Quando a chuva passar\n\nPra quê falar\nSe voc...,pt,Ivete Sangalo,Pop; Axé; Romântico,313.0,4.4,/ivete-sangalo/
4,/ivete-sangalo/,Sorte Grande,/ivete-sangalo/sorte-grande.html,A minha sorte grande foi você cair do céu\nMin...,pt,Ivete Sangalo,Pop; Axé; Romântico,313.0,4.4,/ivete-sangalo/
...,...,...,...,...,...,...,...,...,...,...
378983,/clegg-johnny/,The Waiting,/clegg-johnny/the-waiting.html,Chorus\nHere we stand waiting on the plain\nDa...,en,Johnny Clegg,World Music; Gospel/Religioso,34.0,0.0,/clegg-johnny/
378984,/clegg-johnny/,Too Early For The Sky,/clegg-johnny/too-early-for-the-sky.html,I nearly disappeared into the mouth of a croco...,en,Johnny Clegg,World Music; Gospel/Religioso,34.0,0.0,/clegg-johnny/
378985,/clegg-johnny/,Warsaw 1943 (I Never Betrayed The Revolution),/clegg-johnny/warsaw-1943-i-never-betrayed-the...,"Amambuka, amambuka azothengisa izwe lakithi, i...",en,Johnny Clegg,World Music; Gospel/Religioso,34.0,0.0,/clegg-johnny/
378986,/clegg-johnny/,When The System Has Fallen,/clegg-johnny/when-the-system-has-fallen.html,Sweat in the heat for days on end\nwaiting for...,en,Johnny Clegg,World Music; Gospel/Religioso,34.0,0.0,/clegg-johnny/


In [8]:
# Keep only English lyrics
lyrics = lyrics[lyrics['language'] == 'en']

# Reset index
lyrics = lyrics.reset_index(drop = True)

lyrics.head()

,ALink,SName,SLink,Lyric,language,Artist,Genres,Songs,Popularity,Link
0,/ivete-sangalo/,Careless Whisper,/ivete-sangalo/careless-whisper.html,I feel so unsure\nAs I take your hand and lead...,en,Ivete Sangalo,Pop; Axé; Romântico,313.0,4.4,/ivete-sangalo/
1,/ivete-sangalo/,Could You Be Loved / Citação Musical do Rap: S...,/ivete-sangalo/could-you-be-loved-citacao-musi...,"Don't let them fool, ya\nOr even try to school...",en,Ivete Sangalo,Pop; Axé; Romântico,313.0,4.4,/ivete-sangalo/
2,/ivete-sangalo/,Cruisin' (Part. Saulo),/ivete-sangalo/cruisin-part-saulo.html,"Baby, let's cruise, away from here\nDon't be c...",en,Ivete Sangalo,Pop; Axé; Romântico,313.0,4.4,/ivete-sangalo/
3,/ivete-sangalo/,Easy,/ivete-sangalo/easy.html,"Know it sounds funny\nBut, I just can't stand ...",en,Ivete Sangalo,Pop; Axé; Romântico,313.0,4.4,/ivete-sangalo/
4,/ivete-sangalo/,For Your Babies (The Voice cover),/ivete-sangalo/for-your-babies-the-voice-cover...,You've got that look again\nThe one I hoped I ...,en,Ivete Sangalo,Pop; Axé; Romântico,313.0,4.4,/ivete-sangalo/


In [9]:
# Separate the genres into distinct genres and replicate the song samples
separated_genre_df = pd.DataFrame()
S_Link = []
gen = []

for i in lyrics.index:
    SLink = lyrics['SLink'][i]
    if type(lyrics['Genres'][i]) != str:
        continue
    elif ';' in lyrics['Genres'][i]:
        g_list = lyrics['Genres'][i].split(';')
        for item in g_list:
            S_Link.append(SLink)
            gen.append(item.strip())
    else:
        S_Link.append(SLink)
        gen.append(lyrics['Genres'][i].strip())

separated_genre_df['SLink'] = S_Link
separated_genre_df['DistinctGenre'] = gen
separated_genre_df

,SLink,DistinctGenre
0,/ivete-sangalo/careless-whisper.html,Pop
1,/ivete-sangalo/careless-whisper.html,Axé
2,/ivete-sangalo/careless-whisper.html,Romântico
3,/ivete-sangalo/could-you-be-loved-citacao-musi...,Pop
4,/ivete-sangalo/could-you-be-loved-citacao-musi...,Axé
...,...,...
443292,/clegg-johnny/warsaw-1943-i-never-betrayed-the...,Gospel/Religioso
443293,/clegg-johnny/when-the-system-has-fallen.html,World Music
443294,/clegg-johnny/when-the-system-has-fallen.html,Gospel/Religioso
443295,/clegg-johnny/woman-be-my-country.html,World Music


In [10]:
# Remove the genres with values less than 10000 so we can have the top 15 genres
value_counts = separated_genre_df['DistinctGenre'].value_counts()
value_counts_df = value_counts.reset_index()
value_counts_df.columns = ['Value', 'Count']
value_counts_df = value_counts_df[value_counts_df['Count'] >= 10000]
value_counts_df

,Value,Count
0,Rock,63717
1,Pop,33571
2,Pop/Rock,26360
3,Heavy Metal,21759
4,Hip Hop,20902
5,Indie,20134
6,Romântico,17947
7,Hard Rock,17250
8,Rap,17114
9,R&B,15581


In [11]:
# Merge the dataframes based on SLink to incorporate the distinct genres
separated_genre_df = pd.merge(lyrics, separated_genre_df, on = 'SLink')
separated_genre_df = pd.merge(value_counts_df, separated_genre_df, right_on = 'DistinctGenre', left_on = 'Value')
separated_genre_df

,Value,Count,ALink,SName,SLink,Lyric,language,Artist,Genres,Songs,Popularity,Link,DistinctGenre
0,Rock,63717,/elvis-presley/,Can't Help Falling In Love,/elvis-presley/cant-help-falling-in-love.html,"Wise men say, only fools rush in\nBut I can't ...",en,Elvis Presley,Rockabilly; Romântico; Rock,759.0,23.1,/elvis-presley/,Rock
1,Rock,63717,/elvis-presley/,Suspicious Minds,/elvis-presley/suspicious-minds.html,We're caught in a trap\nI can't walk out\nBeca...,en,Elvis Presley,Rockabilly; Romântico; Rock,759.0,23.1,/elvis-presley/,Rock
2,Rock,63717,/elvis-presley/,Always On My Mind,/elvis-presley/always-on-my-mind.html,"Maybe I didn't treat you,\nQuite as good as I ...",en,Elvis Presley,Rockabilly; Romântico; Rock,759.0,23.1,/elvis-presley/,Rock
3,Rock,63717,/elvis-presley/,Love Me Tender,/elvis-presley/love-me-tender.html,"Love me tender, love me sweet,\nNever let me g...",en,Elvis Presley,Rockabilly; Romântico; Rock,759.0,23.1,/elvis-presley/,Rock
4,Rock,63717,/elvis-presley/,My Way,/elvis-presley/my-way.html,And now the end is near\nAnd so I face the fin...,en,Elvis Presley,Rockabilly; Romântico; Rock,759.0,23.1,/elvis-presley/,Rock
...,...,...,...,...,...,...,...,...,...,...,...,...,...
314128,Country,10630,/true-blood/,Strange Love,/true-blood/strange-love.html,Strange love\nCutest thing I ever seen\nStrang...,en,True Blood,Trilha Sonora; Country,22.0,0.0,/true-blood/,Country
314129,Country,10630,/true-blood/,Swampblood,/true-blood/swampblood.html,"Way down in Toxarcana, I was ten years old,\nI...",en,True Blood,Trilha Sonora; Country,22.0,0.0,/true-blood/,Country
314130,Country,10630,/true-blood/,Take Me Home,/true-blood/take-me-home.html,In the sweet light of the valley\nwhen the sun...,en,True Blood,Trilha Sonora; Country,22.0,0.0,/true-blood/,Country
314131,Country,10630,/true-blood/,The Golden State,/true-blood/the-golden-state.html,You are the hole in my head\nI am the pain in ...,en,True Blood,Trilha Sonora; Country,22.0,0.0,/true-blood/,Country


In [12]:
# Drop all columns apart from lyrics and distinct genre to prepare data for the model
final_df = separated_genre_df[['Lyric', 'DistinctGenre']]

# Keep only examples from the genre 'Country' for training the model
final_df = final_df[final_df['DistinctGenre'] == 'Country']

# Keep only n = 1000 samples
final_df = final_df.sample(n = 1000, random_state = 7)

# Reset Index
final_df = final_df.reset_index(drop = True)

final_df

,Lyric,DistinctGenre
0,She dropped her boy at school on time\nOne les...,Country
1,Hotel room of doom\nI can't find a clue\nConfu...,Country
2,Up on the white veranda\nShe wears a necktie a...,Country
3,Verse 1:\nE\nLord knows i am peaceful when i'm...,Country
4,Come on now Willie don't look at me that way\n...,Country
...,...,...
995,Melinda was mine\n'til the time\nThat I found ...,Country
996,[Florida Georgia Line:]\nEvery time I look in ...,Country
997,Watch out when they start to twist\nHold tight...,Country
998,\n\nSomewhere Over The Rainbow...Way Up High\n...,Country


# Text Pre-Processing

In [13]:
def clean_text(text):
    # Make the lyrics lowercase
    text = text.lower()
    
    # Remove '\n' from the text
    text = text.replace('\n', ' ')

    # Remove punctuation from the text
    translation_table = str.maketrans("", "", string.punctuation)
    text = text.translate(translation_table)

    # Tokenize the text
    tokens = word_tokenize(text)

    # Remove stopwords from the tokens
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word.lower() not in stop_words]

    # Join the tokens back into sentences
    text = ' '.join(tokens)

    return text

In [14]:
tokens = []
for text in final_df['Lyric']:
    tokens.append(clean_text(text))

final_df['Tokens'] = tokens
final_df.head()

,Lyric,DistinctGenre,Tokens
0,She dropped her boy at school on time\nOne les...,Country,dropped boy school time one less worry mind wo...
1,Hotel room of doom\nI can't find a clue\nConfu...,Country,hotel room doom cant find clue confusion broke...
2,Up on the white veranda\nShe wears a necktie a...,Country,white veranda wears necktie panama hat passpor...
3,Verse 1:\nE\nLord knows i am peaceful when i'm...,Country,verse 1 e lord knows peaceful im left alone e ...
4,Come on now Willie don't look at me that way\n...,Country,come willie dont look way know like paperback ...


In [15]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import LambdaCallback
import random
import sys

In [16]:
# Combine all lyrics into a single string
text = ' '.join(final_df['Tokens'].tolist())

# Create a mapping from unique characters to indices
chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [20]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

# Replace this with your list of preprocessed lyrics
lyrics = final_df['Tokens'].tolist()

# Tokenize words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lyrics)
total_words = len(tokenizer.word_index) + 1

# Create input sequences and labels
input_sequences = []
for line in lyrics:
    tokenized_line = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(tokenized_line)):
        n_gram_sequence = tokenized_line[:i+1]
        input_sequences.append(n_gram_sequence)

# Pad sequences for uniform length (max sequence length = 50)
max_sequence_length = 50  # Updated max sequence length
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre')

# Create predictors and labels
X, y = input_sequences[:, :-1], input_sequences[:, -1]
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

# Build the model
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_length-1))
model.add(LSTM(100))
model.add(Dense(total_words, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Training loop with batch processing and within-epoch print statements
seed_text = "Small town boy"
next_words = 50

In [28]:
epoch = 20
if epoch == 20:
    print(f"Epoch {epoch}/{20}")
    
    model = load_model('country_lyric_generator_model.h5')

    model.fit(X, y, epochs=1, batch_size=128, verbose=1, callbacks=[early_stopping], validation_split=0.1)

    model.save('country_lyric_generator_model.h5')
    print('---MODEL SAVED---')

    # Generate lyrics for the specified seed text after each epoch
    generated_sequence = seed_text
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([generated_sequence])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_length-1, padding='pre')
        predicted_probs = model.predict(token_list, verbose=0)

        # Get the index of the word with the highest probability
        predicted_index = np.argmax(predicted_probs[0])

        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted_index:
                output_word = word
                break

        generated_sequence += " " + output_word

    print(f"After epoch {epoch}: {generated_sequence}")

Epoch 20/20
780/780 [==============================] - 323s 412ms/step - loss: 3.5905 - accuracy: 0.3654 - val_loss: 7.9409 - val_accuracy: 0.1322
---MODEL SAVED---
After epoch 20: Small town boy man way back home small town southern man well im goin back town two kinds cherries two kinds fairies two kinds babies two kinds babies two kinds babies two trucks cherries two kinds babies two kinds babies two kinds babies two kinds babies black trucks yeah yeah know im goin


In [29]:
# Generate lyrics for the specified seed text after training
n = np.random.randint(1, 1000)
#ref_text = final_df['Tokens'][n]
#seed_text = ref_text[:5]
seed_text = 'Small town boy'
next_words = 45
model = load_model('country_lyric_generator_model.h5')

generated_sequence = seed_text
for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([generated_sequence])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_length-1, padding='pre')
    predicted_probs = model.predict(token_list, verbose=0)

    # Get the index of the word with the highest probability
    predicted_index = np.argmax(predicted_probs[0])

    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted_index:
            output_word = word
            break

    generated_sequence += " " + output_word

#print('Reference text:\t', ref_text)
print('Generated text:\t', generated_sequence)

Generated text:	 Small town boy man way back home small town southern man well im goin back town two kinds cherries two kinds fairies two kinds babies two kinds babies two kinds babies two trucks cherries two kinds babies two kinds babies two kinds babies two kinds babies black trucks


In [26]:
from nltk.translate.bleu_score import sentence_bleu
from rouge import Rouge

def calculate_bleu(reference, candidate):
    reference_tokens = [reference.split()]
    candidate_tokens = candidate.split()
    return sentence_bleu(reference_tokens, candidate_tokens)

def calculate_rouge(reference, candidate):
    rouge = Rouge()
    scores = rouge.get_scores(candidate, reference)
    return scores[0]['rouge-1']['f'], scores[0]['rouge-2']['f'], scores[0]['rouge-l']['f']

# Example usage:
bleu_score = calculate_bleu(ref_text, generated_sequence)
rouge_1, rouge_2, rouge_l = calculate_rouge(ref_text, generated_sequence)

print(f"BLEU Score: {bleu_score}")
print(f"ROUGE-1 Score: {rouge_1}, ROUGE-2 Score: {rouge_2}, ROUGE-L Score: {rouge_l}")


BLEU Score: 8.793278923262055e-156
ROUGE-1 Score: 0.08450704047609604, ROUGE-2 Score: 0.020408161432736528, ROUGE-L Score: 0.08450704047609604


c:\Users\nidhi\OneDrive\Documents\USA\Stevens\Academics\CS584_NLP\Project\venv\Lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\Users\nidhi\OneDrive\Documents\USA\Stevens\Academics\CS584_NLP\Project\venv\Lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
